# 22강. 딥네트워크, 서로 뭐가 다른 거죠?

## 22-9. VGG-16
### MNIST나 CIFAR100 이미지셋 분류기 모델을 VGG로 바꿔보기

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# CIFAR100 데이터셋 가져오기
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0 # 정규화

print("x_train:", len(x_train), "x_test:", len(x_test))

x_train: 50000 x_test: 10000


In [3]:
img_input = keras.Input(shape=(32, 32, 3))

x = keras.layers.Conv2D(16, 3, activation='relu')(img_input)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=img_input, outputs=predictions)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               295168

In [ ]:
# 모델 학습!! 
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1)

cs231n
- linear classifier : 어떤 w를 쓰는 것이 가장 좋을까?
w를 정량화할 방법이 필요 -> 손실함수 사용(최적화)
x(이미지), y(레이블:1~10사이) -> loss를 최소화
 1. multi-class SVM : 10개의 카테고리이므로 좀 더 다양화 필요
 (p.12)올바른 카테고리 정답 스코어가 올바르지 않은 카테고리 스코어보다 높으면 loss는 0
 cat loss : 2.9(3.2-1.7-5.1+1(margin))
 frog : 12.9
 전체 트레이닝셋 로스 : 로스 평균
 로스 최솟값 = 0
 (p.24)벡터라이즈로 바꿔 순회 줄이기
 (p.25)다른 w도 존재 - 이것 역시도 loss = 0 
 (p.26)2w : margin 도 2배
 (p.28)테스트데이터에 분류기를 사용하여 성능을 보기 때문에 w loss=0인 것을 선택하는 것은 모순.
 (p.33)data loss & regularization loss 더 단순한 것을 선택, regularization lambda를 고려해주어야. 더 복잡해지지 않도록
 (p.34)L2 regularization, L1 regularization, max norm regularization : panelty를 부여해서 완벽하게 맞지 않도록 만들어줌.
 w1, w2 linear classifier에서는 같지만 L2에서는 w2를 더 선호.
 L1은 w의 0개 개수에 따라 복잡도 정의

 2. soft classifier
 (p.41)모든 확률의 합 = 1, 정답클래스확률 = 1에 가까운 값 / loss = -log(P(정답클래스)) 얼마나 구린지 보기 위한 것이기 떄문에 -를 붙임.
 (p.47) soft min=0 / max=무한대
 완벽하게 구분했다면 loss = 0 (컴퓨터는 무한대 계산을 못해서 0으로 떨어지지는 않는다.) 유한 정밀도로는 최댓값 최솟값에 도달할 수 없다.
 
 - softmax의 중요성 : loss를 구할때 유용하니까. cross entropy(불확실성 계산)와 loss함수로 계산하기 위해 softmax로 계산하기 쉽게 숫자 변형. / SVM에서는 cross entropy 불가
 정답으로 가정했기 때문에 t(k)=1로 두고 계산.
 
 두개의 스코어 해석 방법이 좀 다르다. svm: 정답과 정답이 아닌 스코어 간의 마진에 신경. 그래서 어떻게해도 값이 변하지 않음. / softmax확률을 구해서 정답 스코어에 더 신경. 언제나 다 더했을 때 값을 1로. 최대한 정답스코어에 숫자를 넣으려고 할 것.
 성능차이는 크지 않음.
 
 (p.56)optimization(최적화) : 
 (p.57)random search : 임의의 지점에서 시작해서 찾아나가는 방법. 
 (p.59)Follow the slope : 지역적 특징을 이용하기. 왔다갔다. / slope : 어떤 함수에 대한 미분값. x는 벡터이기 때문에 다변수로 확장. gradient는 편도함수들의 기울기. gradient중요. 파라미터 벡터를 업데이트할 떄 사용. 
 (p.61)유한차분법 사용. -> 로스값 조금 감소. 계속 반복. 계속 조금씩 감소. 이런 방식으로 구하지 않음. 너무 많은 값...
 (P.71) 수식으로 나타내서 한번에 계산. 디버깅에 좋다.
 (p.71)Gradient Dscend : 우선 w 임의의 값으로 초기화. loss와 gradient계산해서 조금씩 이동. learning rate 중요한 하이퍼파라미터 중 하나. 빨간 영역 찾아서.
 learning rate나 regularization이 minimum을 찾기 위해 이곳저곳 디뎌보는 것. local minimum에 빠지지 않도록.
 (p.76) SGD : loss gradient의 합. 미니배치로 예시. -> w 업데이트.
 미니배치. 한쪽 방향으로 바라보고 나아가는 것을 여러개 설정.
 (p.81) color histogram : 특징 변환. 해당하는 색의 양동이에 넣고 세는 것. 간단한 특징벡터. 이미지 전반적인 색깔. 색의 분포를 통해 이미지를 식별할 수 있는 방법이 존재한다.
 (p.82)HOG : edge로. 이미지 전반적인 edge
 (p.83)bag of words : 단어의 발생 빈도. 이미지들을 임의대로 조각. 다양한 군집 만들어내기.(=codebook) / encode images
 (p.84) pipline 트레이닝 동안에는 linear classification만. 가중치 전체를 한꺼번에 학습.